In [7]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(f"device is {device}")

device is cuda:0


# Without AutoGrad

In [8]:
dtype = torch.float
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(1000):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 643.7643432617188
199 3.650358200073242
299 0.03006388433277607
399 0.0005329026025719941
499 6.709125591441989e-05
599 2.3765674995956942e-05
699 1.3311190741660539e-05
799 8.765143320488278e-06
899 6.598308573302347e-06
999 5.13531358592445e-06


# With AutoGrad

In [9]:
dtype = torch.float
N, D_in, H, D_out = 64, 1000, 100, 10
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(1000):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss)

    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

99 tensor(490.3634, device='cuda:0', grad_fn=<SumBackward0>)
199 tensor(2.9283, device='cuda:0', grad_fn=<SumBackward0>)
299 tensor(0.0269, device='cuda:0', grad_fn=<SumBackward0>)
399 tensor(0.0005, device='cuda:0', grad_fn=<SumBackward0>)
499 tensor(6.5122e-05, device='cuda:0', grad_fn=<SumBackward0>)
599 tensor(2.2485e-05, device='cuda:0', grad_fn=<SumBackward0>)
699 tensor(1.2482e-05, device='cuda:0', grad_fn=<SumBackward0>)
799 tensor(8.1998e-06, device='cuda:0', grad_fn=<SumBackward0>)
899 tensor(5.9813e-06, device='cuda:0', grad_fn=<SumBackward0>)
999 tensor(4.6024e-06, device='cuda:0', grad_fn=<SumBackward0>)


# Pytorch package: nn